In [3]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import time

In [44]:
import time
item_ids = 1911084
item_info = {
    "item_id": [],
    "bedrooms": [],
    "bathrooms": [],
    "name": [],
    "numberOfRooms": [],
    "price": [],
    "priceCurrency": [],
    "priceCurrency": [],
    "floorSize": [],
    "description": [],
    "address_country": [],
    "address": []
        
}
#цикл менялся на каждом этапе (за раз по 2-3 тысячи наблюдений парсилось)
for item_id in range(1909408,1910000):
    time.sleep(0.5)
    url = 'https://tranio.com/canada/adt/%d/' % item_id
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    if  '<div class="ap-features-val">' not in soup.prettify() or len(soup.find_all('div', attrs={'class': 'ap-features-val'}))<2:
        continue
    elif soup.find('h1').text == 'This listing has been removed' or soup.find_all('div', attrs={'class': 'ap-features-title'})[0].text=='Total area' or soup.find_all('div', attrs={'class': 'ap-features-title'})[0].text=='Land area'  :
        continue
    else:
        main_info = soup.find_all('script', attrs={ 'type':"application/ld+json"})[1].text
        main_info_js = json.loads(main_info)
        item_info["item_id"] += [item_id]
        item_info["bedrooms"] += [soup.find_all('div', attrs={'class': 'ap-features-val'})[0].text]
        item_info["bathrooms"] += [soup.find_all('div', attrs={'class': 'ap-features-val'})[1].text]
        item_info["name"] += [main_info_js.get('name')]
        item_info["numberOfRooms"] += [main_info_js.get('numberOfRooms')]
        item_info["price"] += [main_info_js.get('price')]
        item_info["priceCurrency"] += [main_info_js.get('priceCurrency')]
        item_info["floorSize"] += [main_info_js.get('floorSize')]
        item_info["description"] += [main_info_js.get('description')]
        item_info["address_country"] += [main_info_js.get('address')['addressCountry']['name']]
        item_info["address"] += [main_info_js.get('address')['addressLocality']]
       
    
item_info_df4 = pd.DataFrame(item_info)
item_info_df4.head()

,item_id,bedrooms,bathrooms,name,numberOfRooms,price,priceCurrency,floorSize,description,address_country,address
0,1909408,2,2,"Apartment – Toronto, Ontario, Canada",2,635000,USD,"{'@type': 'QuantitativeValue', 'value': '', 'u...",2 Bedroom\n842 Sq Ft Sweet Suite\nIn This Lowr...,Canada,Toronto
1,1909409,2,2,"Apartment – Toronto, Ontario, Canada",2,634000,USD,"{'@type': 'QuantitativeValue', 'value': '', 'u...",Brand New 2 Bedrooms Condo Located In Heart Of...,Canada,Toronto
2,1909410,2,2,"Apartment – Toronto, Ontario, Canada",2,630000,USD,"{'@type': 'QuantitativeValue', 'value': '', 'u...",Spacious Corner Unit With A 2 Bedroom/2 Full B...,Canada,Toronto
3,1909411,2,1,"Apartment – Toronto, Ontario, Canada",2,618000,USD,"{'@type': 'QuantitativeValue', 'value': '', 'u...",West Harbour Ii With South Facing Luxury! 1 Be...,Canada,Toronto
4,1909412,3,2,"Apartment – North York, Toronto, Ontario, Canada",3,617000,USD,"{'@type': 'QuantitativeValue', 'value': '', 'u...",Enjoy Living In Luxury At The Very Popular Gra...,Canada,North York


In [42]:
s=item_info_df4.loc[item_info_df4['address_country']=='Canada']

In [43]:
s.to_csv('Toronto_final_19.csv')